# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
#drop unnecessary columns
new_df=df.drop(columns='timestamp')

It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [9]:
#! pip install scikit-surprise
#from scikit import surprise
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_df,reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [10]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [11]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [12]:
## Perform a gridsearch with SVD
params = {'n_factors' :[20,50,100],
         'reg_all':[0.02,0.05,0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)


In [13]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8698579511559721, 'mae': 0.6688090638923673}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [14]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)

In [15]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97063188, 0.97158671, 0.97121994, 0.9697047 , 0.97994134]))
('test_mae', array([0.75063567, 0.75022214, 0.75028394, 0.75077272, 0.75456508]))
('fit_time', (0.5089995861053467, 0.5139992237091064, 0.5320003032684326, 0.5040006637573242, 0.49899959564208984))
('test_time', (0.8880000114440918, 0.8960003852844238, 0.8920001983642578, 0.8930304050445557, 0.8790328502655029))
-----------------------
0.9726169127569803


In [16]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [17]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87699573, 0.88282518, 0.87717276, 0.8771004 , 0.87392546]))
('test_mae', array([0.66805801, 0.67225455, 0.67153661, 0.67021552, 0.66717217]))
('fit_time', (0.6360063552856445, 0.6419899463653564, 0.6389691829681396, 0.632004976272583, 0.631005048751831))
('test_time', (1.386993408203125, 1.3529999256134033, 1.3650298118591309, 1.3209939002990723, 1.3399994373321533))


0.8776039060218567

Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peak at what information we have about them.

In [18]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [19]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [20]:

svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [21]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=3.084460896943118, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step to go let's create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [22]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list
        

In [25]:
user_rating = movie_rater(df_movies,4,'Comedy')

      movieId                                              title  \
8051    98697  Money Money Money (L'aventure, c'est l'aventur...   

                genres  
8051  Adventure|Comedy  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId               title                  genres
5140     8241  Masterminds (1997)  Action|Comedy|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
     movieId                   title        genres
538      635  Family Thing, A (1996)  Comedy|Drama
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                title  genres
3332     4511  High Spirits (1988)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                      title                     genres
4060     5785  Jackass: The Movie (2002)  Action|Comedy|Documentary
How do you rate this movie on a scale of 1-5, press n if 

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [24]:
user_rating

[{'userId': 1000, 'movieId': 3114, 'rating': '5'},
 {'userId': 1000, 'movieId': 7293, 'rating': '4'},
 {'userId': 1000, 'movieId': 2792, 'rating': '4n'},
 {'userId': 1000, 'movieId': 575, 'rating': '3n'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [26]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [27]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [28]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))


In [29]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies,key=lambda x:x[1],reverse=True)

In [30]:
ranked_movies

[(750, 4.634192140308329),
 (7361, 4.556530507307317),
 (922, 4.553306558331265),
 (318, 4.539570329615454),
 (2959, 4.531957002487054),
 (1104, 4.5306538539202545),
 (904, 4.510799433342675),
 (1204, 4.506148858844063),
 (1193, 4.504969417110741),
 (50, 4.5025585740779634),
 (951, 4.499047990746266),
 (2324, 4.475319618847076),
 (930, 4.470930884067338),
 (1945, 4.464114346104314),
 (1136, 4.462467066402559),
 (1225, 4.4609525454426056),
 (4973, 4.4593717480173165),
 (1217, 4.4570001893769255),
 (3468, 4.456738279333749),
 (1228, 4.450797024966972),
 (1089, 4.447796421074027),
 (1276, 4.446410601598928),
 (1617, 4.444905148021502),
 (38061, 4.4393282466974044),
 (4011, 4.436202799844407),
 (1252, 4.428855693478752),
 (56782, 4.428720844801436),
 (933, 4.426891823783782),
 (1258, 4.426427312462461),
 (2542, 4.421377355239025),
 (1212, 4.4096633248889265),
 (527, 4.40894420203157),
 (2360, 4.403210798824829),
 (2160, 4.402557750988827),
 (1198, 4.399053570107773),
 (2329, 4.398110299027

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [31]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  4909    Eternal Sunshine of the Spotless Mind (2004)
Name: title, dtype: object 

Recommendation #  3 :  704    Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
Name: title, dtype: object 

Recommendation #  4 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  5 :  2226    Fight Club (1999)
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.